In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    read_loinc_df,
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    read_full_med_df,
    get_visualization_subtitle,
    apply_theme,
    get_anonymousid_color_maps,
    get_siteid_anonymous_map,
    get_anonymousid_color_maps_none_pediatric
)
from web import for_website

alt.data_transformers.disable_max_rows() # Allow using rows more than 5000

In [ ]:
DATA_AS_OF='2020-08-03'
NUM_SITES = len(list(get_anonymousid_color_maps_none_pediatric().keys()))

In [ ]:
original_df = pd.read_csv("../data/Figure_678_med_plot_meta_metafor.csv", sep=",", header=0)

siteid_map = get_siteid_anonymous_map()
original_df.siteId = original_df.siteId.apply(lambda x: 'pool' if x == 'pool' else siteid_map[x])

original_df = original_df[original_df.p.notna()]

original_df

In [ ]:
SITE_NAMES = list(get_anonymousid_color_maps_none_pediatric().keys())
SITE_COLORS = list(get_anonymousid_color_maps_none_pediatric().values())

In [ ]:
df = original_df.copy()

# rename columns
df = df.rename(columns={
    'siteId': 'siteid',
    'time': 'before_or_since',
    'medname': 'med'
})

# rename categories
df.before_or_since = df.before_or_since.apply(lambda x: 'Before admission' if x == 'before_admission' else 'After admission')

df

In [ ]:
MEDICATIONS = df['med'].unique().tolist()
MEDICATIONS

In [ ]:
df_agg = df.copy() 

# filter rows
df_agg = df_agg[df_agg.siteid == 'pool']

# remove columns we don't use
df_agg = df_agg.drop(columns=['siteid'])

df_agg

In [ ]:
dropdown = alt.binding_select(options=MEDICATIONS)
selection = alt.selection_single(fields=['med'], bind=dropdown, name='Medication', init={'med': MEDICATIONS[0]})

tick_size = 40

base = (
    alt.Chart(df_agg)
        .mark_bar(size=tick_size)
        .encode(
            x=alt.Y('before_or_since:N', scale=alt.Scale(domain=['Before admission', 'After admission'])),
            y=alt.Y('ci_95L:Q', title='Pooled mean (CI)'),
            y2=alt.Y2('ci_95U:Q'),
            color=alt.Color(
                'before_or_since:N', 
                title=None, 
                scale=alt.Scale(
                    domain=['Before admission', 'After admission'], 
                    range=['#DC3A11', '#3366CC']
                ),
                legend=None
            ),
            stroke=alt.Color('before_or_since:N', legend=None),
            strokeWidth=alt.value(1),
            tooltip=[
                alt.Tooltip("before_or_since", title="Before or Since"),
                alt.Tooltip("mean", title="Mean", format=".2f"),
                alt.Tooltip("ci_95L", title="CI (.95 Lower)", format=".2f"),
                alt.Tooltip("ci_95U", title="CI (.95 Upper)", format=".2f")
            ]
        )
)

mean = alt.Chart(df_agg).mark_tick(size=tick_size, thickness=2).encode(
    x=alt.X('before_or_since:N', title=None),
    opacity=alt.value(1),
    color=alt.value('white'),
    y=alt.Y('mean:Q')
)

subtitle = get_visualization_subtitle(data_release=DATA_AS_OF, with_num_sites=False) + f'  |  {NUM_SITES} Sites'

plot = alt.layer(
    base, mean
).add_selection(
    selection
).transform_filter(
    selection
).properties(
    title={
        "text": ["Meta-Analysis for Risk Difference"],
        "subtitle": [
            subtitle,
            '',
            '*Risk Difference = ',
            'Risk of Being Severe with Medication –', 
            'Risk of Being Severe without Medication',
        ],
        "subtitleColor": "gray",
        "anchor": "middle",
    },
    width=350, height=400
)

plot = apply_theme(plot).configure_axis(
    labelAngle=0,
    #
    labelFontSize=14,
    labelFontWeight=300,
    titleFontSize=16,
    titleFontWeight=300,
    titlePadding=16
)

for_website(plot, "1.1.Med", "Medication risk difference")

plot

## Forest plot of the risk difference (RD) of being severe among the patients with the medication and among the patients without the medication before admission.  

In [ ]:
df_before = df.copy() 

# filter rows
df_before = df_before[df_before.before_or_since == 'Before admission']
df_before = df_before[df_before.siteid != 'pool']

df_before

In [ ]:
def render_forest(data, before_or_since):
    dropdown = alt.binding_select(options=MEDICATIONS)
    selection = alt.selection_single(fields=['med'], bind=dropdown, name='Medication', init={'med': MEDICATIONS[0]})

    tick_size = 12

    base = (
        alt.Chart(data)
            .mark_bar(size=tick_size)
            .encode(
                y=alt.Y('siteid:N'),
                x=alt.X('ci_95L:Q', title='Pooled mean (CI)'),
                x2=alt.X2('ci_95U:Q'),
                color=alt.Color(
                    'siteid:N',
                    title=None, 
                    scale=alt.Scale(
                        domain=SITE_NAMES, 
                        range=SITE_COLORS
                    ),
                    legend=None
                ),
                # stroke=alt.Color('siteid:N', legend=None),
                # strokeWidth=alt.value(1),
                tooltip=[
                    alt.Tooltip("before_or_since", title="Before or Since"),
                    alt.Tooltip("mean", title="Mean", format=".2f"),
                    alt.Tooltip("ci_95L", title="CI (.95 Lower)", format=".2f"),
                    alt.Tooltip("ci_95U", title="CI (.95 Upper)", format=".2f")
                ]
            )
    )

    mean = alt.Chart(data).mark_tick(size=tick_size, thickness=3).encode(
        y=alt.Y('siteid:N', title=None),
        opacity=alt.value(1),
        color=alt.value('white'),
        # stroke=alt.value('white'),
        # strokeWidth=alt.value(2),
        x=alt.X('mean:Q')
    )

    p_val = alt.Chart(data).mark_text(size=tick_size, thickness=3).encode(
        y=alt.Y('siteid:N', title=None, axis=None),
        color=alt.value('black'),
        text=alt.Text('p:Q', format=".2f"),
    ).transform_filter(
        selection
    ).properties(
        width=70,
        title={
            'text': 'p value',
            'orient': 'top',
            'fontSize': 14
        }
    )

    subtitle = get_visualization_subtitle(data_release=DATA_AS_OF, with_num_sites=False) + f'  |  {NUM_SITES} Sites',

    plot = alt.layer(
        base, mean
    ).add_selection(
        selection
    ).transform_filter(
        selection
    ).properties(
        width=450, height=600
    )

    plot = apply_theme((plot | p_val).resolve_scale(y='shared')).configure_axis(
        labelAngle=0,
        #
        labelFontSize=14,
        labelFontWeight=300,
        titleFontSize=16,
        titleFontWeight=300,
        titlePadding=16
    ).properties(
        title={
            "text": [f"Risk Difference Being Severe {before_or_since} Admission"],
            "subtitle": subtitle,
            "subtitleColor": "gray",
            "anchor": "middle",
        },
    )
    
    for_website(plot, "1.1.Med", f"Medication Risk Difference {before_or_since} Admission")
    
    return plot

In [ ]:
render_forest(df_before, 'Before')

In [ ]:
df_after = df.copy()

# filter rows
df_after = df_after[df_after.before_or_since == 'After admission']
df_after = df_after[df_after.siteid != 'pool']

df_after

In [ ]:
render_forest(df_after, 'Since')

In [ ]:
medlab_df = pd.read_csv("../data/Figure_9_plot_diuretic_creatine.csv", sep=",", header=0)

medlab_df = medlab_df.rename(columns={
    'site': 'siteid',
    'Proportion of Patients with Diuretic among ever severe': 'Ever severe',
    'Proportion of Patients with Diuretic among never severe': 'Never severe',
    'Max Creatine Over 14 days': 'max_creatine'
})

medlab_df = pd.melt(
    medlab_df,
    id_vars=['scale', 'siteid', 'max_creatine'],
    value_vars=['Ever severe', 'Never severe'],
    var_name='severity',
    value_name='prop_diuretic'
)

original_medlab_df = medlab_df[medlab_df.scale == 'original']
log_medlab_df = medlab_df[medlab_df.scale == 'log']

medlab_df

In [ ]:
original = (
    alt.Chart(original_medlab_df)
        .mark_point(size=50)
        .encode(
            x=alt.Y('prop_diuretic:Q', title='Proportion of patients with diuretic'),
            y=alt.Y('max_creatine:Q', title='Max creatine over 14 days'),
            color=alt.Color(
                'severity:N', 
                title=None, 
                scale=alt.Scale(
                    domain=['Ever severe', 'Never severe'], 
                    range=['#DC3A11', '#3366CC']
                )
            ),
            # stroke=alt.Color('before_or_since:N', legend=None),
            # strokeWidth=alt.value(1),
            # tooltip=[
            #    alt.Tooltip("before_or_since", title="Before or Since"),
            #    alt.Tooltip("mean", title="Mean", format=".2f"),
            #    alt.Tooltip("ci_95L", title="CI (.95 Lower)", format=".2f"),
            #    alt.Tooltip("ci_95U", title="CI (.95 Upper)", format=".2f")
            # ]
        )
).properties(
    title={
        'text': 'Original scale',
        'fontSize': 16
    }
)

log = (
    alt.Chart(log_medlab_df)
        .mark_point(size=50)
        .encode(
            x=alt.Y('prop_diuretic:Q', title='Proportion of patients with diuretic'),
            y=alt.Y('max_creatine:Q', title='Max creatine over 14 days'),
            color=alt.Color(
                'severity:N', 
                title=None, 
                scale=alt.Scale(
                    domain=['Ever severe', 'Never severe'], 
                    range=['#DC3A11', '#3366CC']
                )
            ),
            # stroke=alt.Color('before_or_since:N', legend=None),
            # strokeWidth=alt.value(1),
            # tooltip=[
            #    alt.Tooltip("before_or_since", title="Before or Since"),
            #    alt.Tooltip("mean", title="Mean", format=".2f"),
            #    alt.Tooltip("ci_95L", title="CI (.95 Lower)", format=".2f"),
            #    alt.Tooltip("ci_95U", title="CI (.95 Upper)", format=".2f")
            # ]
        )
).properties(
    title={
        'text': 'Log scale',
        'fontSize': 16
    }
)

subtitle = get_visualization_subtitle(data_release=DATA_AS_OF, with_num_sites=False) + f'  |  {NUM_SITES} Sites'

plot = (
    original | log
).properties(
    title={
        "text": ["Diuretic vs. Creatine"],
        "subtitle": [
            subtitle
        ],
        "subtitleColor": "gray",
        "anchor": "middle",
    }
)

plot = apply_theme(plot).configure_axis(
    labelAngle=0,
    labelFontSize=14,
    labelFontWeight=300,
    titleFontSize=16,
    titleFontWeight=300,
    titlePadding=16
)

for_website(plot, "1.1.Med", "Diuretic vs. Creatine")

plot